In [2]:
import pandas as pd
import numpy as np

np.random.seed(7)

In [3]:
def genX(n):
    np.random.seed(7)
    x1 = np.random.uniform(-5, 5, n)
    x2 = np.random.uniform(-2, 2, n)
    x0 = np.ones(n)
    df = pd.DataFrame({'x0': x0, 'x1': x1, 'x2': x2})
    return x0, x1, x2, df

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
def gen_data(n):
    np.random.seed(7)
    x0, x1, x2, x_dataframe = genX(n)

    linear_y = 3 * x0 + 1 * x1 - 2 * x2 + np.random.normal(0, 0.05, n)

    y = np.random.binomial(1, sigmoid(linear_y))

    return x_dataframe, y

In [5]:
def predict_row(row, coefficients):

    pred_terms = np.multiply(row, coefficients)
    yhat = np.sum(pred_terms)
    return 1 / (1 + np.exp(-yhat))

In [6]:
def mse(y, yhat):
    return np.mean((y - yhat)**2)

def nll(y, yhat):
    return -np.mean(y * np.log(yhat) + (1 - y) * np.log(1 - yhat))

In [27]:
n = 1000
X, y = gen_data(n)

# Random guess

In [8]:
def true_model():
    print('True model:')
    y_pred = np.zeros(n)

    for row in range(n):
        y_pred[row] = predict_row(np.array(X.iloc[row]), np.array([3,1,-2]))

    print(f'MSE: {mse(y, y_pred)}')
    print(f'NLL: {nll(y, y_pred)}')

In [9]:
y_pred = np.zeros(n)

for row in range(n):
    y_pred[row] = predict_row(np.array(X.iloc[row]), np.array([0,0.5,1]))

print(f'MSE: {mse(y, y_pred)}')
print(f'NLL: {nll(y, y_pred)}')

print()
true_model()

MSE: 0.2996565695865238
NLL: 0.9081213170722178

True model:
MSE: 0.06831581573421616
NLL: 0.21844118029674345


# Logistic Regression

The partial derivative of the NLL with respect to a coefficient ($ \beta_k\ $) is given by:

$$
\frac{\partial \text{NLL}}{\partial \beta_k} = (\hat{y} - y) \cdot x_k
$$


This derivative indicates how much $ \beta_k  $ should change to reduce the error.

##### Update Rule:
Using the learning rate ($\eta$), the update rule for the coefficients is:

$$
\beta_k \gets \beta_k - \eta \cdot \frac{\partial \text{NLL}}{\partial \beta_k}
$$

Substituting the derivative:

$$
\beta_k \gets \beta_k - \eta \cdot (\hat{y} - y) \cdot x_k
$$


In [10]:
# for sake of testing
i = 1
coefs = np.zeros(3) # (beta_0,beta_1,beta_2)
l_rate = 0.01

In [11]:
# Predict the outcome for row i
yhat_i = predict_row(np.array(X.iloc[i]), coefs)

# Update each coefficient
for k in range(len(coefs)):
    coefs[k] = coefs[k] - l_rate * (yhat_i - y[i]) * X.iloc[i, k]

In [12]:
coefs

array([0.005     , 0.01399594, 0.00052306])

# Stochastic gradient descent

In [13]:
coefs = np.zeros(3) # (beta_0,beta_1,beta_2)
l_rate = 0.01
epochs = 10

for _ in range(epochs):
    i = np.arange(n)
    np.random.shuffle(i)
    for index in i:
        # Predict the outcome for row i
        yhat_i = predict_row(np.array(X.iloc[index]), coefs)

        # Update each coefficient
        for k in range(len(coefs)):
            coefs[k] = coefs[k] - l_rate * (yhat_i - y[index]) * X.iloc[index, k]

coefs

array([ 2.91031708,  1.06186663, -1.7851285 ])

# Convert our estimator into a function!

In [17]:
def train(X , y, l_rate, epochs):
    n = X.shape[0]
    coefs = np.zeros(X.shape[1]) # (beta_0,beta_1,beta_2)

    for epoch in range(epochs):
        yhats = np.zeros(n)
        i = np.arange(n)
        np.random.shuffle(i)
        for index in i:
            # Predict the outcome for row i
            yhat_i = predict_row(X[index, :], coefs)
            yhats[index] = yhat_i

            # Update each coefficient
            for k in range(len(coefs)):
                coefs[k] = coefs[k] - l_rate * (yhat_i - y[index]) * X[index, k]

        print()
        print(f'Epoch: {epoch+1}')
        print(f'MSE: {mse(y, yhats)}')
        print(f'NLL: {nll(y, yhats)}')

    return coefs

In [28]:
coefs = train(np.array(X), y, 0.01, 50)
coefs


Epoch: 1
MSE: 0.11263013736413799
NLL: 0.36802017870647363

Epoch: 2
MSE: 0.07754586065326635
NLL: 0.26249982704373986

Epoch: 3
MSE: 0.072276854916161
NLL: 0.23989746755172156

Epoch: 4
MSE: 0.06994573356201068
NLL: 0.2298914150944504

Epoch: 5
MSE: 0.06904666369499703
NLL: 0.22505467755652445

Epoch: 6
MSE: 0.06894033514491528
NLL: 0.22231721138972438

Epoch: 7
MSE: 0.06888994071212937
NLL: 0.22156562895248452

Epoch: 8
MSE: 0.06866697562401922
NLL: 0.22059103779985192

Epoch: 9
MSE: 0.06855474975977552
NLL: 0.21984699834999005

Epoch: 10
MSE: 0.06899042192323145
NLL: 0.21987740920681068

Epoch: 11
MSE: 0.0677493272336344
NLL: 0.21810797552721506

Epoch: 12
MSE: 0.0685432797683098
NLL: 0.218805213067784

Epoch: 13
MSE: 0.06860037734506745
NLL: 0.21806116793966077

Epoch: 14
MSE: 0.06866977681666168
NLL: 0.21877129509944485

Epoch: 15
MSE: 0.06816273347709582
NLL: 0.217211367242172

Epoch: 16
MSE: 0.0684131254195128
NLL: 0.21735741958602228

Epoch: 17
MSE: 0.06818689318575598
NLL: 0.

array([ 3.31964693,  1.11147701, -1.99262977])

In [35]:
# compare to ligistic regression package
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X, y)

print(f'Coefficients: {clf.coef_}')

print(f'Intercept: {clf.intercept_}')


Coefficients: [[-0.02595394  1.11737484 -1.93152413]]
Intercept: [3.23600159]
